In [1]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import json
from tqdm import tqdm
import sys
import os
import time
import re
from src.settings import NODES_DIR

UNKNOWN = 'Nieznane'
LOGIN = 'a'
PASSWORD = 'b'

In [2]:
nodes = pd.read_csv(NODES_DIR)
authors_data = zip(nodes['id'].to_list(), nodes['label'].to_list())
options = Options()
options.headless = False
driver = webdriver.Firefox("./", options=options)
# driver.get(f"https://dona.pwr.edu.pl/szukaj/default.aspx?nrewid=428140")

wait = WebDriverWait(driver, 20)
entry = {}
options = Options()


all_entries = []
driver.get('https://dona.pwr.edu.pl/szukaj/')
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadButton_login")))
driver.find_element_by_css_selector("#RadButton_login").click()

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadTextBox_uzytkownik")))
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadTextBox_haslo")))
driver.find_element_by_css_selector("#RadTextBox_uzytkownik").send_keys(LOGIN)
driver.find_element_by_css_selector("#RadTextBox_haslo").send_keys(PASSWORD)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadButton_zaloguj")))
driver.find_element_by_css_selector("#RadButton_zaloguj").click()
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadDock_formularze_C_RadSearchBox1_Input")))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6f21e52a-a0f7-4f75-8da6-037c165ca216", element="42bffe49-45e5-4abf-9ba5-663bd128c2a6")>

In [3]:
driver.get(f"https://dona.pwr.edu.pl/szukaj/default.aspx?nrewid=428140")
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadDock1_info_autora_C_RadLabel_pytanie1")))
author_info = driver.find_element_by_css_selector("#RadDock1_info_autora_C_RadLabel_pytanie1").text
data = author_info.split("\n")

department_info = data[0]
department_tokens = department_info.rsplit(', ', maxsplit=1)
if len(department_tokens) > 1:
    department = department_tokens[1].strip(']')
else:
    department = UNKNOWN

In [4]:
if len(data) > 1:
    orcid_data = data[1]
    match = re.search("ORCID:\s*(.+)\s*", orcid_data)
    if match:
        orcid = match.group(1)
    else:
        orcid = UNKNOWN
else:
    orcid = UNKNOWN

In [5]:
if len(data) > 2:
    disciplines = data[2]
    match = re.search("dyscypliny:\s*(.+)\s*", disciplines)
    if match:
        disciplines = match.group(1).split(', ')
    else:
        disciplines = []
else:
    disciplines = []

In [6]:
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadDock1_info_autora_C_RadLabel_liczba_prac")))
work_and_impact_factor_data = driver.find_element_by_css_selector("#RadDock1_info_autora_C_RadLabel_liczba_prac").text
tokens = work_and_impact_factor_data.split('\n')
if len(tokens) > 3:
    impact_factor_line = tokens[3]
    impact_factor = impact_factor_line.split(':')[1].strip()
else:
    impact_factor = 0
    

In [7]:
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadTabStrip_dane_wyszukane")))
button_row = driver.find_element_by_css_selector("#RadTabStrip_dane_wyszukane").text.split('\n')
publications_number = '0'
citations = '0'
supervisorships = '0'
for button in button_row:
    match = re.match(r"(.+):\s*(.+)", button)
    if match:
        name = match.group(1)
        value = match.group(2)
        if name == 'Prace':
            publications_number = value.strip()
        elif name == 'Promotorstwa':
            supervisorships = value.strip()
        elif name == 'Cytowania':
            citations = value.strip()

In [8]:
entry = {
    'department': department,
    'orcid': orcid,
    'disciplines': disciplines,
    'publications': publications_number,
    'citations': citations,
    'impact_factor': impact_factor,
    'supervisorships': supervisorships
}